In [6]:
# LSTM for sequence classification in the IMDB dataset
import numpy as np
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Convolution1D, Flatten, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard

In [18]:
from nltk.corpus import stopwords
import string

In [10]:
stopword = stopwords.words('english')

In [2]:
# Using keras to load the dataset with the top_words
top_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [2]:
dataset = pd.read_csv('train_F3WbcTw.csv')

In [5]:
data = dataset[['text','sentiment']]

In [11]:
data['clean_text'] = dataset['text'].apply(lambda x: x.lower())

/home/aptus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
data['clean_text'] = data['clean_text'].apply(lambda x : ' '.join([word.translate(str.maketrans('', '', string.punctuation)) for word in x.split() if word not in stopword]))

/home/aptus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
data.head()

,text,sentiment,clean_text
0,Autoimmune diseases tend to come in clusters. ...,2,autoimmune diseases tend come clusters. gileny...
1,I can completely understand why you’d want to ...,2,"completely understand you’d want try it. but, ..."
2,Interesting that it only targets S1P-1/5 recep...,2,interesting targets s1p-1/5 receptors rather 1...
3,"Very interesting, grand merci. Now I wonder wh...",2,"interesting, grand merci. wonder lemtrada ocre..."
4,"Hi everybody, My latest MRI results for Brain ...",1,"hi everybody, latest mri results brain cervica..."


In [3]:
# Pad the sequence to the same length
max_review_length = 1600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [4]:
# Using embedding from Keras
embedding_vecor_length = 300
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:

# Convolutional model (3x conv, flatten, 2x dense)
model.add(Convolution1D(64, 3, padding='same'))
model.add(Convolution1D(32, 3, padding='same'))
model.add(Convolution1D(16, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:

# Log to tensorboard
tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.fit(X_train, y_train, epochs=3, callbacks=[tensorBoardCallback], batch_size=64)



Instructions for updating:
Use tf.cast instead.
Epoch 1/3
25000/25000 [==============================] - 391s 16ms/step - loss: 0.3745 - acc: 0.8277
Epoch 2/3
25000/25000 [==============================] - 405s 16ms/step - loss: 0.1642 - acc: 0.9396
Epoch 3/3
25000/25000 [==============================] - 416s 17ms/step - loss: 0.0590 - acc: 0.9802


In [8]:
# Evaluation on the test set
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


Accuracy: 84.64%
